In [71]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.utils as vutils
from torch.utils.data import DataLoader
from PIL import Image
import numpy as np


In [72]:

import torchvision
from torchvision import transforms

def load_dataset(folder_path):
    transform = transforms.Compose([
        transforms.Resize((32, 32)),  # Resize images to 32x32
        transforms.ToTensor(),       # Convert images to tensors
        transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
    ])
    dataset = torchvision.datasets.ImageFolder(folder_path, transform=transform)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)
    return dataloader

In [73]:
# Define the Generator
class Generator(nn.Module):
    def __init__(self, latent_dim=100):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 3 * 32 * 32),  # Output 3-channel image (RGB)
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        return img.view(-1, 3, 32, 32)  # Reshape to (batch, 3, 32, 32)


# Define the Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(3 * 32 * 32, 512),  # Adjust input size to match flattened image size (3072)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)  # Flatten the image
        return self.model(img_flat)

In [106]:
# Function to initialize weights
def initialize_weights(model):
    classname = model.__class__.__name__
    if classname.find("Conv") != -1:
        nn.init.normal_(model.weight.data, mean=0.0, std=0.02)
    elif classname.find("BatchNorm") != -1:
        nn.init.normal_(model.weight.data, mean=1.0, std=0.02)
        nn.init.constant_(model.bias.data, 0)

# Train the GAN
def train_gan(dataloader, num_epochs=100, latent_dim=100, lr=0.002):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    generator = Generator(latent_dim).to(device)
    discriminator = Discriminator().to(device)

    # Apply weight initialization
    generator.apply(initialize_weights)
    discriminator.apply(initialize_weights)

    loss_function = nn.BCELoss()
    optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.00005, betas=(0.5, 0.999))

    for epoch in range(num_epochs):
        for i, (real_images, _) in enumerate(dataloader):
            real_images = real_images.to(device)
            batch_size = real_images.size(0)

            # Real and fake labels
            real_labels = torch.ones(batch_size, 1).to(device)
            fake_labels = torch.zeros(batch_size, 1).to(device)

            
            optimizer_D.zero_grad()
            outputs_real = discriminator(real_images)
            loss_real = loss_function(outputs_real, real_labels)

            z = torch.randn(batch_size, latent_dim).to(device)
            fake_images = generator(z)
            outputs_fake = discriminator(fake_images.detach())
            loss_fake = loss_function(outputs_fake, fake_labels)

            loss_D = loss_real + loss_fake
            loss_D.backward()
            optimizer_D.step()

            for j in range(2):
                optimizer_G.zero_grad()
                z = torch.randn(batch_size, latent_dim).to(device)
                fake_images = generator(z)

                outputs = discriminator(fake_images)
                loss_G = loss_function(outputs, real_labels)

                retain_graph = (j == 0)
                loss_G.backward(retain_graph=retain_graph)
                optimizer_G.step()

        print(f"Epoch [{epoch+1}/{num_epochs}] | Loss D: {loss_D.item():.4f} | Loss G: {loss_G.item():.4f}")

        if (epoch + 1) % 10 == 0:
            vutils.save_image(fake_images, f"generated_epoch_{epoch+1}.png", normalize=True)

    torch.save(generator.state_dict(), "generator.pth")


In [107]:
# Generate New Images
def generate_images(generator_path, num_images=5, latent_dim=100):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    generator = Generator(latent_dim).to(device)
    generator.load_state_dict(torch.load(generator_path, map_location=device))
    generator.eval()

    z = torch.randn(num_images, latent_dim).to(device)
    with torch.no_grad():
        fake_images = generator(z)  # Shape: (num_images, channels, height, width)

    for i, img in enumerate(fake_images):
        img = img.cpu().numpy()  # Convert tensor to numpy
        img = ((img + 1) / 2 * 255).astype(np.uint8)  # Normalize to [0, 255]

        if img.shape[0] == 1:  # Grayscale Image (Shape: 1, H, W)
            img = img.squeeze(0)  # Remove channel dimension
            Image.fromarray(img, mode="L").save(f"generated_{i}.png")
        elif img.shape[0] == 3:  # RGB Image (Shape: 3, H, W)
            img = np.transpose(img, (1, 2, 0))  # Convert from (C, H, W) -> (H, W, C)
            Image.fromarray(img, mode="RGB").save(f"generated_{i}.png")

    print(f"Generated {num_images} images and saved them.")

In [108]:
folder_path = r"D:\college\imp-doc\sem6\GENAI\project\new"
dataloader = load_dataset(folder_path)
train_gan(dataloader)


Epoch [1/100] | Loss D: 1.4266 | Loss G: 0.7201
Epoch [2/100] | Loss D: 1.7262 | Loss G: 0.4325
Epoch [3/100] | Loss D: 1.7003 | Loss G: 0.4858
Epoch [4/100] | Loss D: 1.7342 | Loss G: 0.4646
Epoch [5/100] | Loss D: 1.5375 | Loss G: 0.6278
Epoch [6/100] | Loss D: 1.4734 | Loss G: 0.6464
Epoch [7/100] | Loss D: 1.4303 | Loss G: 0.7099
Epoch [8/100] | Loss D: 1.4677 | Loss G: 0.6934
Epoch [9/100] | Loss D: 1.3264 | Loss G: 0.7511
Epoch [10/100] | Loss D: 1.5875 | Loss G: 0.6860
Epoch [11/100] | Loss D: 1.4282 | Loss G: 0.6949
Epoch [12/100] | Loss D: 1.5668 | Loss G: 0.6434
Epoch [13/100] | Loss D: 1.4925 | Loss G: 0.6388
Epoch [14/100] | Loss D: 1.4785 | Loss G: 0.8118
Epoch [15/100] | Loss D: 1.4137 | Loss G: 0.6992
Epoch [16/100] | Loss D: 1.4166 | Loss G: 0.7258
Epoch [17/100] | Loss D: 1.5295 | Loss G: 0.6772
Epoch [18/100] | Loss D: 1.4050 | Loss G: 0.6997
Epoch [19/100] | Loss D: 1.5533 | Loss G: 0.6522
Epoch [20/100] | Loss D: 1.3906 | Loss G: 0.6782
Epoch [21/100] | Loss D: 1.38

In [109]:
generate_images("generator.pth")

Generated 5 images and saved them.


C:\Users\sameer\AppData\Local\Temp\ipykernel_33180\1683722680.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  generator.load_state_dict(torch.load(generator_path, map_lo

In [110]:
import torch
import torchvision.utils as vutils

# Generate a batch of images
latent_dim = 100
generator = Generator(latent_dim)  # Ensure the generator is loaded
z = torch.randn(1, latent_dim)  # Generate a random noise vector
generated_img = generator(z)

# Save the image
vutils.save_image(generated_img, "generated_image1.png", normalize=True)
print("Image saved as 'generated_image1.png'")

Image saved as 'generated_image1.png'


In [ ]:
!pip install matplotlib
import matplotlib.pyplot as plt

# Assuming you have stored loss values in lists
epochs = range(1, len(loss_G) + 1)  # Adjust based on stored losses

plt.figure(figsize=(10, 5))
plt.plot(epochs, generator_loss, label="Generator Loss (G)", color="blue")
plt.plot(epochs, discriminator_loss, label="Discriminator Loss (D)", color="red")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Generator & Discriminator Loss Over Time")
plt.legend()
plt.grid()
plt.show()


  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   -------------- ------------------------- 2.9/8.1 MB 15.2 MB/s eta 0:00:01
   ------------------------------- -------- 6.3/8.1 MB 16.8 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 16.1 MB/s eta 0:00:00
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 13.8 MB/s eta 0:00:00
Using cached pyparsing-3.2.3-py3-none-any.whl (111 kB)


NameError: name 'generator_loss' is not defined